## **`Paso 5.1`: Creamos el Data flow `df_transform_hospital_admissions` para transformar los datos del archivo `hospital-admissions.csv`**

### **Diagrama Data Flow**

In [ ]:
           SOURCE                            SELECT                       LOOKUP                      SELECT                      SPLIT CONDITIONAL                    JOIN                     PIVOT                    SORT                    SELECT               DERIVED COLUMN                SINK
 ___________________________          ____________________          _________________          _____________________          ________________________           ________________          _______________          _______________          _______________          _______________          _______________
|                           |        |                    |        |                 |        |                     |        |        (Weekly)        |         |                |        |               |        |               |        |               |        |               |        |               |         
|  HospitalAdmissionSource  |--------|  SelectReqdFields  |--------|  LookupCountry  |--------|  SelectReqdFields2  |--------|  SplitDailyFromWeekly  |---------|  JoinWithDate  |--------|  PivotWeekly  |--------|   SortWeekly  |--------|  SelectWeekly |--------| DerivedWeekly |--------|   WeeklySink  |
|___________________________| +      |____________________| +      |_________________| +      |_____________________| +      |________________________| +       |________________| +      |_______________| +      |_______________| +      |_______________| +      |_______________| +      |_______________|
                                                                      |                                                      |                                         PIVOT                     SORT                     SELECT              DERIVED COLUMN                 SINK
        SOURCE                                                        |                                                      |   ________________________          _______________          _______________          _______________          _______________          _______________                                      
 ___________________                                                  |                                                      |  |         (Daily)        |        |               |        |               |        |               |        |               |        |               |         
|                   |-------------------------------------------------'                                                      '--|  SplitDailyFromWeekly  |--------|   PivotDaily  |--------|   SortDaily   |--------|  SelectDaily  |--------|   DailySink   |--------|   DailySink   |    
|   CountrySource   |                                                                                                           |________________________| +      |_______________| +      |_______________| +      |_______________| +      |_______________| +      |_______________|                                           
|___________________|    

       SOURCE                   DERIVED COLUMN                 AGGREGATE
 _________________          ______________________          ________________
|                 |        |                      |        |                |   
|  DimDateSource  |--------|  DeriveECDCYearWeek  |--------|   AggDimDate   |
|_________________| +      |______________________| +      |________________|   

[![p1033.png](https://i.postimg.cc/gjyTM9vQ/p1033.png)](https://postimg.cc/K3zfzHpD)

### **1. Crear un nuevo dataset de origen que haga referencia al archivo `hospital-admissions.csv`, llamado  `ds_raw_hospital_admission`**

#### **Paso 1: Crear un nuevo dataset**

1. **Seleccionar la pestaña "Author"**: En el menú de la izquierda dentro de tu Data Factory, haz clic en la pestaña **Author** (Autor).
2. **Seleccionar "Datasets"**: En el panel de **Author**, haz clic en **Datasets**.
3. **Hacer clic en los tres puntos**: Junto a la opción **Datasets**, haz clic en los tres puntos (más opciones).
4. **Seleccionar "New dataset"**: En el menú desplegable, selecciona **New dataset** para crear un nuevo dataset.

[![p743.png](https://i.postimg.cc/26Z4n5zc/p743.png)](https://postimg.cc/R6v6B4JK)
[![p926.png](https://i.postimg.cc/7ZRskMkG/p926.png)](https://postimg.cc/rDSCG4zM)

#### **Paso 2: Configurar el nuevo dataset**

1. **Elegir tipo de dataset**: En la ventana de selección, busca y selecciona `Azure Data Lake Storage Gen2` como el tipo de datastore.
2. **Hacer clic en "Continue"**: Después de seleccionar `Azure Data Lake Storage Gen2`, haz clic en **Continue** para proceder.

[![p927.png](https://i.postimg.cc/GpsMh8LM/p927.png)](https://postimg.cc/nspK3LQ7)

#### **Paso 3: Configurar el formato del dataset**

1. **Seleccionar el formato**: En el campo de selección de formato, elige `DelimitedText`.
2. **Hacer clic en "Continue"**: Después de seleccionar el formato, haz clic en **Continue**.

[![p928.png](https://i.postimg.cc/d3JnB58X/p928.png)](https://postimg.cc/jwkzqQ3Q)

#### **Paso 4: Configurar las propiedades del dataset**

- Crearemos un nuevo dataset **ds_ecdc_raw_csv_dl**, que hará referencia hacia nuestro ADLS y esta vez tendrá un formato de tipo `DelimitedText`. Hara referencia al archivo `cases_deaths.csv` (que no existe aún) que se almacenará en el contenedor `raw` y en el directorio `ecdc` (tampoco existe aún). No importaremos el schema, dado que el archivo aún no existe, no tiene datos.

1. **Nombre del dataset**: En el campo de nombre, ingresa `ds_raw_hospital_admission`.
2. **Linked Service**: En el menú desplegable, selecciona el Linked Service que creaste previamente, `ls_adls_covidreportingdl`.
3. **File path**: Indiquemos la ruta `raw/ecdc` e ingresa `hospital_admissions.csv`.
4. **First row as header**: Activar la casilla.
5. **Import schema**: Selecciona `From connection/store` para importar un schema.
6. **Guardar el dataset**: Haz clic en el botón **OK** para almacenar el nuevo dataset.

[![p929.png](https://i.postimg.cc/JnVpyrtR/p929.png)](https://postimg.cc/LqDtwMzW)

#### **Paso 5: Revisar el dataset**

1. **Revisar la lista de datasets**: Deberías ver tu nuevo dataset `ds_raw_hospital_admissions` en la lista de datasets. Puedes hacer clic en él para revisar su configuración.
2. **Revisar el schema**: Podemos visualizar el schema de datos del archivo `hospital_admissions.csv`.

[![p930.png](https://i.postimg.cc/xCvtMwbM/p930.png)](https://postimg.cc/34wXHb1x)
[![p931.png](https://i.postimg.cc/brLgF1gv/p931.png)](https://postimg.cc/MfM1nMhk)

### **2. Crear un nuevo dataset de origen que haga referencia al archivo `dim_date.csv`, llamado  `ds_dim_date_lookup`**

#### **Paso 1: Crear un nuevo dataset**

1. **Seleccionar la pestaña "Author"**: En el menú de la izquierda dentro de tu Data Factory, haz clic en la pestaña **Author** (Autor).
2. **Seleccionar "Datasets"**: En el panel de **Author**, haz clic en **Datasets**.
3. **Hacer clic en los tres puntos**: Junto a la opción **Datasets**, haz clic en los tres puntos (más opciones).
4. **Seleccionar "New dataset"**: En el menú desplegable, selecciona **New dataset** para crear un nuevo dataset.

[![p743.png](https://i.postimg.cc/26Z4n5zc/p743.png)](https://postimg.cc/R6v6B4JK)
[![p958.png](https://i.postimg.cc/8CBWTMJB/p958.png)](https://postimg.cc/NyFL4yx5)

#### **Paso 2: Configurar el nuevo dataset**

1. **Elegir tipo de dataset**: En la ventana de selección, busca y selecciona `Azure Data Lake Storage Gen2` como el tipo de datastore.
2. **Hacer clic en "Continue"**: Después de seleccionar `Azure Data Lake Storage Gen2`, haz clic en **Continue** para proceder.

[![p959.png](https://i.postimg.cc/LX9P55jd/p959.png)](https://postimg.cc/SXPRtyZV)

#### **Paso 3: Configurar el formato del dataset**

1. **Seleccionar el formato**: En el campo de selección de formato, elige `DelimitedText`.
2. **Hacer clic en "Continue"**: Después de seleccionar el formato, haz clic en **Continue**.

[![p960.png](https://i.postimg.cc/J0gHpQvm/p960.png)](https://postimg.cc/JHNhnjRd)

#### **Paso 4: Configurar las propiedades del dataset**

1. **Nombre del dataset**: En el campo de nombre, ingresa `ds_dim_date_lookup`.
2. **Linked Service**: En el menú desplegable, selecciona el Linked Service que creaste previamente, `ls_adls_covidreportingdl`.
3. **File path**: Indiquemos el contenedor `lookup` e ingresa `dim_date.csv`.
4. **First row as header**: Activar la casilla.
5. **Import schema**: Selecciona `From connection/store` para importar un schema.
6. **Guardar el dataset**: Haz clic en el botón **OK** para almacenar el nuevo dataset.

[![p961.png](https://i.postimg.cc/8csrdYyt/p961.png)](https://postimg.cc/1Vhtsv1q)

#### **Paso 5: Revisar el dataset**

1. **Revisar la lista de datasets**: Deberías ver tu nuevo dataset `ds_dim_date_lookup` en la lista de datasets. Puedes hacer clic en él para revisar su configuración.
2. **Revisar el schema**: Podemos visualizar el schema de datos del archivo `dim_date.csv`.

[![p962.png](https://i.postimg.cc/Y9dWW7xv/p962.png)](https://postimg.cc/dkT1bzpF)
[![p963.png](https://i.postimg.cc/VLrCd30N/p963.png)](https://postimg.cc/0bvQ4Xfg)

### **3. Crear un nuevo dataset de destino que haga referencia a la ruta `processed/ecdc/hospital_admissions_weekly`, llamado  `ds_processed_hospital_admissions_weekly`**

Creamos un nuevo dataset de destino **ds_processed_hospital_admission_weekly** que haga referencia a la ruta **processed/ecdc/hospital_admissions_weekly** de nuestro ADLS. No se indicó un nombre de archivo.

#### **Paso 1: Crear un nuevo dataset**

1. **Seleccionar la pestaña "Author"**: En el menú de la izquierda dentro de tu Data Factory, haz clic en la pestaña **Author** (Autor).
2. **Seleccionar "Datasets"**: En el panel de **Author**, haz clic en **Datasets**.
3. **Hacer clic en los tres puntos**: Junto a la opción **Datasets**, haz clic en los tres puntos (más opciones).
4. **Seleccionar "New dataset"**: En el menú desplegable, selecciona **New dataset** para crear un nuevo dataset.

[![p743.png](https://i.postimg.cc/26Z4n5zc/p743.png)](https://postimg.cc/R6v6B4JK)
[![p1011.png](https://i.postimg.cc/Dfbj1fDV/p1011.png)](https://postimg.cc/hhc1BncM)

#### **Paso 2: Configurar el nuevo dataset**

1. **Elegir tipo de dataset**: En la ventana de selección, busca y selecciona `Azure Data Lake Storage Gen2` como el tipo de datastore.
2. **Hacer clic en "Continue"**: Después de seleccionar `Azure Data Lake Storage Gen2`, haz clic en **Continue** para proceder.

[![p1012.png](https://i.postimg.cc/7Y8B9885/p1012.png)](https://postimg.cc/Wt86py4V)

#### **Paso 3: Configurar el formato del dataset**

1. **Seleccionar el formato**: En el campo de selección de formato, elige `DelimitedText`.
2. **Hacer clic en "Continue"**: Después de seleccionar el formato, haz clic en **Continue**.

[![p1013.png](https://i.postimg.cc/dVn4Jh7C/p1013.png)](https://postimg.cc/0zMpf205)

#### **Paso 4: Configurar las propiedades del dataset**

1. **Nombre del dataset**: En el campo de nombre, ingresa `ds_processed_hospital_admission_weekly`.
2. **Linked Service**: En el menú desplegable, selecciona el Linked Service que creaste previamente, `ls_adls_covidreportingdl`.
3. **File path**: Indiquemos el contenedor `processed` e ingresa la ruta `ecdc/hospital_admissions_weekly`.
4. **First row as header**: Activar la casilla.
5. **Import schema**: Selecciona `None` para no importar un schema, dado que no existe aún el archivo.
6. **Guardar el dataset**: Haz clic en el botón **OK** para almacenar el nuevo dataset.

[![p1014.png](https://i.postimg.cc/kXQTh1qf/p1014.png)](https://postimg.cc/Wh4md7Dk)

#### **Paso 5: Revisar el dataset**

1. **Revisar la lista de datasets**: Deberías ver tu nuevo dataset `ds_processed_hospital_admissions_weekly` en la lista de datasets. Puedes hacer clic en él para revisar su configuración.

[![p1015.png](https://i.postimg.cc/8zTZ8RCD/p1015.png)](https://postimg.cc/3dbXmDpb)

### **4. Crear un nuevo dataset de destino que haga referencia a la ruta `processed/ecdc/hospital_admissions_daily`, llamado  `ds_processed_hospital_admissions_daily`**

Debemos crear un nuevo dataset de destino **ds_processed_hospital_admissions_daily** que haga referencia a la ruta **processed/ecdc/hospital_admissions_daily** de nuestro ADLS. No se indicó un nombre de archivo.

#### **Paso 1: Crear un nuevo dataset**

1. **Seleccionar la pestaña "Author"**: En el menú de la izquierda dentro de tu Data Factory, haz clic en la pestaña **Author** (Autor).
2. **Seleccionar "Datasets"**: En el panel de **Author**, haz clic en **Datasets**.
3. **Hacer clic en los tres puntos**: Junto a la opción **Datasets**, haz clic en los tres puntos (más opciones).
4. **Seleccionar "New dataset"**: En el menú desplegable, selecciona **New dataset** para crear un nuevo dataset.

[![p743.png](https://i.postimg.cc/26Z4n5zc/p743.png)](https://postimg.cc/R6v6B4JK)
[![p1022.png](https://i.postimg.cc/63hdfSdv/p1022.png)](https://postimg.cc/BLXLqm7S)

#### **Paso 2: Configurar el nuevo dataset**

1. **Elegir tipo de dataset**: En la ventana de selección, busca y selecciona `Azure Data Lake Storage Gen2` como el tipo de datastore.
2. **Hacer clic en "Continue"**: Después de seleccionar `Azure Data Lake Storage Gen2`, haz clic en **Continue** para proceder.

[![p1023.png](https://i.postimg.cc/VvLXGRRH/p1023.png)](https://postimg.cc/XGmZX9tw)

#### **Paso 3: Configurar el formato del dataset**

1. **Seleccionar el formato**: En el campo de selección de formato, elige `DelimitedText`.
2. **Hacer clic en "Continue"**: Después de seleccionar el formato, haz clic en **Continue**.

[![p1024.png](https://i.postimg.cc/SNRWcgjn/p1024.png)](https://postimg.cc/CzyBgHG0)

#### **Paso 4: Configurar las propiedades del dataset**

1. **Nombre del dataset**: En el campo de nombre, ingresa `ds_processed_hospital_admission_daily`.
2. **Linked Service**: En el menú desplegable, selecciona el Linked Service que creaste previamente, `ls_adls_covidreportingdl`.
3. **File path**: Indiquemos el contenedor `processed` e ingresa la ruta `ecdc/hospital_admissions_daily`.
4. **First row as header**: Activar la casilla.
5. **Import schema**: Selecciona `None` para no importar un schema, dado que no existe aún el archivo.
6. **Guardar el dataset**: Haz clic en el botón **OK** para almacenar el nuevo dataset.

[![p1025.png](https://i.postimg.cc/Zn7p1dsY/p1025.png)](https://postimg.cc/JHXsXGWS)

#### **Paso 5: Revisar el dataset**

1. **Revisar la lista de datasets**: Deberías ver tu nuevo dataset `ds_processed_hospital_admissions_daily` en la lista de datasets. Puedes hacer clic en él para revisar su configuración.

[![p1026.png](https://i.postimg.cc/kMfQV0Mr/p1026.png)](https://postimg.cc/F1kd8nJG)

### **5. Crear un nuevo Data Flow llamado `df_transform_hospital_admissions`**

#### **Paso 1: Crear un nuevo Data Flow llamado `df_transform_hospital_admissions` y agregar la actividad `Source`** 

1. **Seleccionar "Data flows"**: En el panel de **Author**, haz clic en **Data flows**.
2. **Hacer clic en los tres puntos**: Junto a la opción **Data flows**, haz clic en los tres puntos (más opciones).
3. **Seleccionar "New data flow"**: En el menú desplegable, selecciona **New data flow** para crear un nuevo Data Flow.
4. Nombrar el Data Flow como `df_transform_hospital_admissions`.

[![p932.png](https://i.postimg.cc/gJBhQ4Rr/p932.png)](https://postimg.cc/DJLmbrxK)
[![p933.png](https://i.postimg.cc/SR29h87d/p933.png)](https://postimg.cc/hQnv8XYQ)

#### **Paso 2: Configurar la actividad `Source` en el Data Flow**

1. Una vez en la página del Data Flow, verás un lienzo en blanco donde puedes agregar actividades. En la parte superior, haz clic en el ícono de **Add Source**.
2. **Pestaña Source Settings**:
   - En **Output stream name**, ingresa `HospitalAdmissionSource`.
   - En **Source type**, selecciona `Dataset`.
   - En **Dataset**, selecciona `ds_raw_hospital_admission`.

2. **Pestaña Projection**:
   - Seleccionamos la opción **Detect data type** para que identifique los tipos de datos correctos para cada campo.

3. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo.

4. **Pestaña Preview**:
   - Previsualizamos los datos.   

[![p934.png](https://i.postimg.cc/Y05Wq9dH/p934.png)](https://postimg.cc/K35z0ZXq)
[![p935.png](https://i.postimg.cc/y6t9TQsb/p935.png)](https://postimg.cc/z3wVzkLC)
[![p936.png](https://i.postimg.cc/sDWWHY2D/p936.png)](https://postimg.cc/ZCT0Cy31)
[![p937.png](https://i.postimg.cc/8zV627Fd/p937.png)](https://postimg.cc/rKQsWFNz)
[![p938.png](https://i.postimg.cc/bwHtpqb7/p938.png)](https://postimg.cc/qNqRGV1w)

#### **Paso 3: Agregar la actividad `Select` para renombrar y eliminar campos**

1. Agregamos la actividad `Select`
2. **Pestaña Select Settings**:
   - En **Output stream name**, ingresa `SelectRequiredFields`.
   - En **Input columns**  eliminamos 1 columna: `url`.
   - En **Input columns**  renombramos 2 columna: `date` a `reported_date` y `year_week` a `reported_year_week`.

3. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo.

4. **Pestaña Data Preview**:
   - Previsualizamos los datos de cada campo.   

[![p939.png](https://i.postimg.cc/5yLBZTVp/p939.png)](https://postimg.cc/8fkfvysr)
[![p940.png](https://i.postimg.cc/TYjj0zQ3/p940.png)](https://postimg.cc/qzRCMZ9P)
[![p941.png](https://i.postimg.cc/c4GMQxRz/p941.png)](https://postimg.cc/NLbHtwP8)
[![p942.png](https://i.postimg.cc/L8NtKjcd/p942.png)](https://postimg.cc/qN6h26BQ)
[![p943.png](https://i.postimg.cc/XYVfzsDc/p943.png)](https://postimg.cc/8j0fFmGj)

#### **Paso 4: Agregar una nueva actividad `Source`**

1. **Pestaña Source Settings**:
   - En **Output stream name**, ingresa `CountrySource`.
   - En **Source type**, selecciona `Dataset`.
   - En **Dataset**, selecciona `ds_country_lookup`.

2. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo.

3. **Pestaña Preview**:
   - Previsualizamos los datos.   

[![p944.png](https://i.postimg.cc/Xqf2wppV/p944.png)](https://postimg.cc/XBvcWNfh)
[![p945.png](https://i.postimg.cc/kXy1G60s/p945.png)](https://postimg.cc/jLLQk5dn)
[![p946.png](https://i.postimg.cc/jSgMQVdJ/p946.png)](https://postimg.cc/0zSGPBS8)

#### **Paso 5: Agregar una nueva actividad `Lookup`**

1. Agregamos la actividad `Lookup`
2. **Pestaña Lookup Settings**:
   - En **Output stream name** ingresa `LookupCountry`.
   - En **Primary stream** selecciona `SelectRequiredFields`.
   - En **Lookup stream** selecciona `CountrySource`.   
   - En **Lookup conditions**: Haremos el match entre el campo `country` de la actividad `SelectRequiredFields` y el campo `country` de la actividad `CountrySource`.

3. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo.

[![p947.png](https://i.postimg.cc/nzjVctMK/p947.png)](https://postimg.cc/5YJWnTBj)
[![p948.png](https://i.postimg.cc/W3BNdfJB/p948.png)](https://postimg.cc/wyk8P222)
[![p949.png](https://i.postimg.cc/zvTJKSCV/p949.png)](https://postimg.cc/jnd0rfkb)

#### **Paso 6: Agregar una nueva actividad `Select` para eliminar una columna** 

1. Agregamos la actividad `Select`
2. **Pestaña Select Settings**:
   - En **Output stream name** ingresa `SelectForSink`.
   - En **Input columns** eliminamos 1 campo: `continent`. El campo duplicado `country` (luego de utilizar el stream **lookup**) no se tomará en cuenta dado que tenemos activadas las casillas de `Skip duplicate`.

3. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo.

[![p950.png](https://i.postimg.cc/DfQQ2Vs3/p950.png)](https://postimg.cc/Bt6PgVMm)
[![p951.png](https://i.postimg.cc/TYRV1kqL/p951.png)](https://postimg.cc/yWbgryGs)
[![p952.png](https://i.postimg.cc/J4PbkbPT/p952.png)](https://postimg.cc/Vd0rybyM)

#### **Paso 7: Agregar una nueva actividad `Split conditional`**

1. Agregamos la actividad `Split conditional`
2. **Pestaña Conditional split Settings**:
   - En **Output stream name** ingresa `SplitDailyFromWeekly`.
   - En **Split condition**:
      - **Stream name**: `Weekly` 
      - **Condition**: `indicator == 'Weekly new hospital admissions per 100k' || indicator == 'Weekly new ICU admissions per 100k'`
      - **Stream name**: `Daily`       

3. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo.

4. **Pestaña Data Preview**:
   - Previsualizar los datos de cada campo.   

[![p953.png](https://i.postimg.cc/6Q9Jmxd6/p953.png)](https://postimg.cc/3kbck6Sz)
[![p954.png](https://i.postimg.cc/xC4w8kBq/p954.png)](https://postimg.cc/KktqsYGy)
[![p955.png](https://i.postimg.cc/BtWR5rS0/p955.png)](https://postimg.cc/zHjd8PHt)
[![p956.png](https://i.postimg.cc/6q4DvWNc/p956.png)](https://postimg.cc/jnKgVr2n)
[![p957.png](https://i.postimg.cc/1tXxy0CL/p957.png)](https://postimg.cc/WDBYnZPn)

#### **Paso 8: Agregar una nueva actividad `Source`**

1. **Pestaña Source Settings**:
   - En **Output stream name**, ingresa `DimDateSource`.
   - En **Source type**, selecciona `Dataset`.
   - En **Dataset**, selecciona `ds_dim_date_lookup`.

2. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo.

3. **Pestaña Data Preview**:
   - Previsualizamos los datos.   

[![p964.png](https://i.postimg.cc/bJfh9S3M/p964.png)](https://postimg.cc/1gB29Xxc)
[![p965.png](https://i.postimg.cc/sxVCtdF4/p965.png)](https://postimg.cc/XGDPCPYr)
[![p966.png](https://i.postimg.cc/26dfcjt5/p966.png)](https://postimg.cc/ftbFksD4)

#### **Paso 9: Agregar una nueva actividad `Derived Column` para agregar un nuevo campo**

Una breve explicación de la función **`LPAD`**:

**LPAD (LEFT-PAD)**: En el primer argumento le indicamos la columna que queremos trabajar, en el segundo parámetro indicamos el total de caracteres que tendrá el valor de la columna y en el tercer parámetro indicamos el caracter que se agregará. 
```plaintext
                                    SELECT LPAD(col, 10, 'v') FROM tabla;

                                    vvvEngland
                                    vvvEngland
                                    vvvEngland
                                    vvvvvWales
                                    vvvEngland
                                    vvvEngland
```

1. Agregamos la actividad `Derived Column`
2. **Pestaña Derived column's Settings**:
   - En **Output stream name**, ingresa `DerivedECDCYearWeek`.
   - En **Columns**:
      - **Column**: `ecdc_year_week`
      - **Expression**: `year + '-W' + lpad(week_of_year,2,'0')`

3. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo.

4. **Pestaña Data Preview**:
   - Previsualizamos los datos.  

[![p967.png](https://i.postimg.cc/d3XnGr9Z/p967.png)](https://postimg.cc/rR1Skd3q)
[![p968.png](https://i.postimg.cc/RVkXF1Fj/p968.png)](https://postimg.cc/KR5npMtN)
[![p969.png](https://i.postimg.cc/44F8CRT4/p969.png)](https://postimg.cc/NLm8TVMZ)
[![p970.png](https://i.postimg.cc/tCZBpvX3/p970.png)](https://postimg.cc/vcyrvzxT)

#### **Paso 10: Agregar una nueva actividad `Aggregate`**

1. Agregamos la actividad `Aggregate`
2. **Pestaña Aggregate Settings**:
   - En **Output stream name**, ingresa `AggDimDate`.
   - En **Group by** agrupamos por la columna **ecdc_year_week**:
      - **Columns**: `ecdc_year_week`
      - **Name as**: `ecdc_year_week`
   - En **Aggregates** podemos agregar utilizando un **MIN** y un **MAX**:
      - **Column**: `week_start_date`
      - **Expression**: `min(date)`     
      - **Column**: `week_end_date`
      - **Expression**: `max(date)`          

3. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo.

4. **Pestaña Data Preview**:
   - Previsualizamos los datos.  

[![p971.png](https://i.postimg.cc/9QTVRBBT/p971.png)](https://postimg.cc/mPbKKCJ2)
[![p972.png](https://i.postimg.cc/bvhPQ265/p972.png)](https://postimg.cc/87txGsgd)
[![p973.png](https://i.postimg.cc/cHDNyksq/p973.png)](https://postimg.cc/Pp8714pK)
[![p974.png](https://i.postimg.cc/TPJ6QYF3/p974.png)](https://postimg.cc/Lq5w8Rmc)
[![p975.png](https://i.postimg.cc/HxWCN114/p975.png)](https://postimg.cc/R3Y838FN)

#### **Paso 11: Agregar una nueva actividad `Join`**

1. Agregamos la actividad `Join`
2. **Pestaña Join Settings**:
   - En **Output stream name**, ingresa `JoinWithDate`.
   - En **Left stream**: `SplitDailyFromWeekly@Weekly`
   - En **Right stream**: `AggDimDate`
   - En **Join type**: `Inner`
   - En **Aggregates** podemos agregar utilizando un **MIN** y un **MAX**:
   - En **Join conditions**: 
      - **Left: SplitDailyFromWeekly@Weekly's column**: `reported_year_week`     
      - **Right: AggDimDate's column**: `ecdc_year_week`      

3. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo.

4. **Pestaña Data Preview**:
   - Previsualizamos los datos. 

[![p976.png](https://i.postimg.cc/FHfbjYf0/p976.png)](https://postimg.cc/FY4JvsVK)
[![p977.png](https://i.postimg.cc/0yLYZ9mF/p977.png)](https://postimg.cc/pmJ5LbsJ)
[![p978.png](https://i.postimg.cc/9fZGnsYM/p978.png)](https://postimg.cc/zLDbLtMZ)
[![p979.png](https://i.postimg.cc/hjn9WM20/p979.png)](https://postimg.cc/sBnB5PdB)

#### **Paso 12: Agregar una nueva actividad `Pivot` llamado `PivotWeekly`**

1. Agregamos la actividad `Pivot`
2. **Pestaña Pivot Settings**:
   - En **Output stream name**, ingresa `PivotWeekly`.
   - En **Input columns**: 
      - **Group by**: Agrupamos 8 columnas: `country`, `country_code_2_digit`, `country_code_3_digit`, `population`, `reported_year_week`, `source`, `week_start_date` y `week_end_date`.
      - **Pivot key**: La columna **Pivot key** sera `indicator`.
        - **Value**: `Weekly new hospital admissions per 100k`
        - **Value**: `Weekly new ICU admissions per 100k`
      - **Pivoted columns**: La **columna pivoteada** será la suma de valores de `value`. Como **expression prefix** colocamos `count`.

3. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo. 

4. **Pestaña Data Preview**:
   - Previsualizamos los datos de cada campo.    

Esto es lo que nos entrega el stream **Join**:

[![p1000.png](https://i.postimg.cc/BnCnh0jN/p1000.png)](https://postimg.cc/pmpvT3Yn)

[![p980.png](https://i.postimg.cc/SsVVgwm9/p980.png)](https://postimg.cc/vggWB2g8)
[![p981.png](https://i.postimg.cc/0yvNdQyb/p981.png)](https://postimg.cc/3dL733pT)
[![p982.png](https://i.postimg.cc/L4CNLKCK/p982.png)](https://postimg.cc/t763GfG2)
[![p983.png](https://i.postimg.cc/zXypkt6L/p983.png)](https://postimg.cc/PPkmX4th)
[![p984.png](https://i.postimg.cc/7LYYdW74/p984.png)](https://postimg.cc/WdCvJSXY)
[![p985.png](https://i.postimg.cc/g0SkWHqZ/p985.png)](https://postimg.cc/WDZPmryN)

#### **Paso 13: Agregar una nueva actividad `Pivot` llamado `PivotDaily`**

1. Agregamos la actividad `Pivot`
2. **Pestaña Pivot Settings**:
   - En **Output stream name**, ingresa `PivotDaily`.
   - En **Input columns**: 
      - **Group by**: Agrupamos 6 columnas: `country`, `country_code_2_digit`, `country_code_3_digit`, `population`, `reported_year_week` y `source`.
      - **Pivot key**: La columna **Pivot key** sera `indicator`.
        - **Value**: `Daily hospital occupancy`
        - **Value**: `Daily ICU occupancy`
      - **Pivoted columns**: La **columna pivoteada** será la suma de valores de `value`. Como **expression prefix** colocamos `count`.

3. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo. 

4. **Pestaña Data Preview**:
   - Previsualizamos los datos de cada campo.    

[![p986.png](https://i.postimg.cc/8Cz2BQ82/p986.png)](https://postimg.cc/hXNZcYts)
[![p987.png](https://i.postimg.cc/90ZKFcT7/p987.png)](https://postimg.cc/dZQ45P5s)
[![p988.png](https://i.postimg.cc/Zqs1w74C/p988.png)](https://postimg.cc/Tym7wcy6)
[![p989.png](https://i.postimg.cc/9M4sKdQ1/p989.png)](https://postimg.cc/sQrnZZSZ)
[![p990.png](https://i.postimg.cc/TwZB6Kyx/p990.png)](https://postimg.cc/qhx1cMXD)
[![p991.png](https://i.postimg.cc/NGHWxkVj/p991.png)](https://postimg.cc/bdqCykp7)

#### **Paso 14: Agregar una nueva actividad `Sort` llamado `SortWeekly`**

1. Agregamos la actividad `Sort`
2. **Pestaña Sort Settings**:
   - En **Output stream name**, ingresa `SortWeekly`.
   - En **Sort conditions**: 
      - **PivotWeekly's column**: `reported_year_week`.
      - **Order**: `Descending`.
      - **PivotWeekly's column**: `country`.
      - **Order**: `Ascending`.      

3. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo. 

4. **Pestaña Data Preview**:
   - Previsualizamos los datos de cada campo.  

[![p992.png](https://i.postimg.cc/QxPKrsmb/p992.png)](https://postimg.cc/N56j13W9)
[![p993.png](https://i.postimg.cc/dVyksXSK/p993.png)](https://postimg.cc/jDRdv8b8)
[![p994.png](https://i.postimg.cc/DfQH4yHC/p994.png)](https://postimg.cc/qgRj90kn)
[![p995.png](https://i.postimg.cc/fTwFcQhM/p995.png)](https://postimg.cc/fkrBsFLP)

#### **Paso 15: Agregar una nueva actividad `Sort` llamado `SortDaily`**

1. Agregamos la actividad `Sort`
2. **Pestaña Sort Settings**:
   - En **Output stream name**, ingresa `SortDaily`.
   - En **Sort conditions**: 
      - **PivotWeekly's column**: `reported_date`.
      - **Order**: `Descending`.
      - **PivotWeekly's column**: `country`.
      - **Order**: `Ascending`.      

3. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo. 

4. **Pestaña Data Preview**:
   - Previsualizamos los datos de cada campo.  

[![p996.png](https://i.postimg.cc/BZj17Xc6/p996.png)](https://postimg.cc/pytTpVSb)
[![p997.png](https://i.postimg.cc/WpxqBTX7/p997.png)](https://postimg.cc/s192SdFB)
[![p998.png](https://i.postimg.cc/VLM01phP/p998.png)](https://postimg.cc/XpjYkH3s)
[![p999.png](https://i.postimg.cc/GmC8pz08/p999.png)](https://postimg.cc/ZCwK7854)

#### **Paso 16: Agregar la actividad `Select` llamado `SelectWeekly` para renombrar columnas**

1. Agregamos la actividad `Select`
2. **Pestaña Select Settings**:
   - En **Output stream name**, ingresa `SelectWeekly`.
   - En **Input columns**  renombramos 4 columna: 
      - `week_start_date` a `reported_week_start_date`
      - `week_end_date` a `reported_week_end_date`
      - `Weekly new hospital admissions per 100k_count` a `new_hospital_occupancy_count`
      - `Weekly new ICU admissions per 100k_count` a `new_icu_occupancy_count`

3. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo.

4. **Pestaña Data Preview**:
   - Previsualizamos los datos de cada campo.   

[![p1001.png](https://i.postimg.cc/44b44G5G/p1001.png)](https://postimg.cc/V5vP43pZ)
[![p1002.png](https://i.postimg.cc/RVgMfTyb/p1002.png)](https://postimg.cc/xcbDVL0K)
[![p1003.png](https://i.postimg.cc/cL6dcnN1/p1003.png)](https://postimg.cc/xXDrfqCB)
[![p1004.png](https://i.postimg.cc/YCVMDCp6/p1004.png)](https://postimg.cc/87h2jDfc)
[![p1005.png](https://i.postimg.cc/TYddps7D/p1005.png)](https://postimg.cc/9zk5kLjm)

#### **Paso 16: Agregar la actividad `Select` llamado `SelectDaily` para renombrar columnas**

1. Agregamos la actividad `Select`
2. **Pestaña Select Settings**:
   - En **Output stream name**, ingresa `SelectDaily`.
   - En **Input columns**  renombramos 2 columna: 
      - `Daily hospital occupancy_count` a `hospital_occupancy_count`
      - `Daily ICU occupancy_count` a `icu_occupancy_count`

3. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo.

4. **Pestaña Data Preview**:
   - Previsualizamos los datos de cada campo.   

[![p1006.png](https://i.postimg.cc/dQWXjSQd/p1006.png)](https://postimg.cc/23Z2zGZ8)
[![p1007.png](https://i.postimg.cc/RVNYsQqY/p1007.png)](https://postimg.cc/0b1ZrSYG)
[![p1008.png](https://i.postimg.cc/HsRhfcyk/p1008.png)](https://postimg.cc/Vd9gtNgx)
[![p1009.png](https://i.postimg.cc/LsM08cvd/p1009.png)](https://postimg.cc/pmqB01gk)
[![p1010.png](https://i.postimg.cc/Z5NMZb8F/p1010.png)](https://postimg.cc/pm2ZJt1p)

#### **Paso 17: Agregar una nueva actividad `Derived Column` llamado `DerivedWeekly` para cambiar tipos de datos**

|Esta actividad será útil en situaciones en las que queramos cambiar el destino de los datos, por ejemplo, guardándolos en una base de datos. Sin embargo, en este caso no es necesaria, ya que los datos se guardarán en un archivo CSV, y en este tipo de archivo todos los valores se almacenan como `String`. A pesar de no ser relevante para este escenario, realizamos este paso para ilustrar en qué casos podría ser útil|
|---|

1. Agregamos la actividad `Derived Column`
2. **Pestaña Derived column's Settings**:
   - En **Output stream name**, ingresa `DerivedWeekly`.
   - En **Columns**:
      - **Column**: `reported_week_start_date`
      - **Expression**: `toDate(reported_week_start_date, 'yyyy-MM-dd')`
      - **Column**: `reported_week_end_date`
      - **Expression**: `toDate(reported_week_end_date, 'yyyy-MM-dd')`
      - **Column**: `population`
      - **Expression**: `toInteger(population)`      

3. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo.

4. **Pestaña Data Preview**:
   - Previsualizamos los datos.  

[![p1074.png](https://i.postimg.cc/wv7LBDfB/p1074.png)](https://postimg.cc/JDCG6HWw)
[![p1075.png](https://i.postimg.cc/RFyfJqyL/p1075.png)](https://postimg.cc/r0Gzvy8D)
[![p1076.png](https://i.postimg.cc/nrmm4ptb/p1076.png)](https://postimg.cc/gLYrd9JN)
[![p1077.png](https://i.postimg.cc/XNB9ScBH/p1077.png)](https://postimg.cc/XX3r958d)

#### **Paso 18: Agregar una nueva actividad `Derived Column` llamado `DerivedDaily` para cambiar tipos de datos**

|Esta actividad será útil en situaciones en las que queramos cambiar el destino de los datos, por ejemplo, guardándolos en una base de datos. Sin embargo, en este caso no es necesaria, ya que los datos se guardarán en un archivo CSV, y en este tipo de archivo todos los valores se almacenan como `String`. A pesar de no ser relevante para este escenario, realizamos este paso para ilustrar en qué casos podría ser útil|
|---|

1. Agregamos la actividad `Derived Column`
2. **Pestaña Derived column's Settings**:
   - En **Output stream name**, ingresa `DerivedDaily`.
   - En **Columns**:
      - **Column**: `population`
      - **Expression**: `toInteger(population)`      

3. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo.

4. **Pestaña Data Preview**:
   - Previsualizamos los datos.  

[![p1078.png](https://i.postimg.cc/NfYFJ16r/p1078.png)](https://postimg.cc/yWQ7W3rV)
[![p1079.png](https://i.postimg.cc/GhQ2ZHKg/p1079.png)](https://postimg.cc/SJnk9Qwc)
[![p1080.png](https://i.postimg.cc/rwwmcMYV/p1080.png)](https://postimg.cc/SJHqfFh3)
[![p1081.png](https://i.postimg.cc/wj4BjZSh/p1081.png)](https://postimg.cc/MnRwsPZT)

#### **Paso 19: Agregar una actividad `Sink` llamado `WeeklySink`** 

1. Agregamos la actividad `Sink`
2. **Pestaña Sink**:
   - En **Output stream name**, ingresa `WeeklySink`.
   - En **Source type**, selecciona `Dataset`.
   - En **Dataset**, selecciona `ds_processed_hospital_admission_weekly`.

3. **Pestaña Settings**:
   - Seleccionamos la opción **Clear folder** para que borre todo en la ruta antes de crear el archivo.
   - En **File name option** seleccionamos `Output to single file`.
   - En **Output to single file** ingresamos `hospital_admissions_weekly.csv`.   

4. **Pestaña Optimize**:
   - Seleccionamos la opción `single partition`. 

5. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo. 

6. **Pestaña Data Preview**:
   - Previsualizamos los datos de cada campo. 

[![p1016.png](https://i.postimg.cc/mZSBrK9t/p1016.png)](https://postimg.cc/dDDzHH4Y)
[![p1017.png](https://i.postimg.cc/DyyF45Q5/p1017.png)](https://postimg.cc/YjVTJ6sm)
[![p1018.png](https://i.postimg.cc/L8GM7pc0/p1018.png)](https://postimg.cc/qNcYh9M8)
[![p1019.png](https://i.postimg.cc/vHqyr93y/p1019.png)](https://postimg.cc/56CD1XhK)
[![p1020.png](https://i.postimg.cc/sgMrwXNh/p1020.png)](https://postimg.cc/nXfgLHZF)
[![p1021.png](https://i.postimg.cc/cJYNGzZ1/p1021.png)](https://postimg.cc/Ff96jD7w)

#### **Paso 20: Agregar una actividad `Sink` llamado `DailySink`** 

1. Agregamos la actividad `Sink`
2. **Pestaña Sink**:
   - En **Output stream name**, ingresa `DailySink`.
   - En **Source type**, selecciona `Dataset`.
   - En **Dataset**, selecciona `ds_processed_hospital_admissions_daily`.

3. **Pestaña Settings**:
   - Seleccionamos la opción **Clear folder** para que borre todo en la ruta antes de crear el archivo.
   - En **File name option** seleccionamos `Output to single file`.
   - En **Output to single file** ingresamos `hospital_admissions_daily.csv`.   

4. **Pestaña Optimize**:
   - Seleccionamos la opción `single partition`. 

5. **Pestaña Inspect**:
   - Verificamos los tipos de datos de cada campo. 

6. **Pestaña Data Preview**:
   - Previsualizamos los datos de cada campo. 

[![p1027.png](https://i.postimg.cc/fRtGYPV7/p1027.png)](https://postimg.cc/VJwZ8DB5)
[![p1028.png](https://i.postimg.cc/mgv0jS9v/p1028.png)](https://postimg.cc/LJVC6LND)
[![p1029.png](https://i.postimg.cc/vBMCFH3X/p1029.png)](https://postimg.cc/CZPcbYyf)
[![p1030.png](https://i.postimg.cc/W40xHJ6h/p1030.png)](https://postimg.cc/675H7qPX)
[![p1031.png](https://i.postimg.cc/76Vdx8R4/p1031.png)](https://postimg.cc/7G5mVcDt)
[![p1032.png](https://i.postimg.cc/sxg8xHFh/p1032.png)](https://postimg.cc/Vr24HRkf)

### **6. Crear el Pipeline de ingesta de datos `pl_process_hospital_admissions_data` para ejecutar el Data flow recien creado**

#### **Paso 1: Crear un nuevo Pipeline**

1. **Seleccionar la pestaña "Author"**: En el menú de la izquierda dentro de tu Data Factory, haz clic en la pestaña **Author** (Autor).
2. **Seleccionar "Pipelines"**: En el panel de Authoring, haz clic en **Pipelines**.
3. **Hacer clic en los tres puntos**: Junto a la opción **Pipelines**, haz clic en los tres puntos (más opciones).
4. **Seleccionar "New pipeline"**: En el menú desplegable, selecciona **New pipeline** para crear un nuevo pipeline.

[![p743.png](https://i.postimg.cc/26Z4n5zc/p743.png)](https://postimg.cc/R6v6B4JK)
[![p1034.png](https://i.postimg.cc/gkfSb20R/p1034.png)](https://postimg.cc/RNLQc9KV)

#### **Paso 2: Configurar y agregar la actividad `Data Flow` el nuevo Pipeline**

1. **Asignar un nombre al Pipeline**: En la parte superior derecha, cambia el nombre del pipeline a `pl_process_hospital_admissions_data`.
2. **Buscar la actividad "Data Flow"**: En el panel de actividades a la izquierda, busca `Data Flow` en la sección de `Move and transform`.
3. **Arrastrar la actividad al canvas**: Arrastra la actividad `Data Flow` al canvas del pipeline.
4. **Pestaña General**:
   - En **Name**, ingresa `Data Flow Transform Hospital Admissions`.
5. **Pestaña Settings**:
   - En  **Data flow** seleccionamos `ds_transform_hospital_admissions`.

[![p1035.png](https://i.postimg.cc/nrbhMcsK/p1035.png)](https://postimg.cc/vghsNytD)
[![p1036.png](https://i.postimg.cc/N0nMqxBz/p1036.png)](https://postimg.cc/yJFB0cJm)

#### **Paso 3: Validar el Pipeline**

1. **Validar la Configuración**: En la parte superior del panel, haz clic en el botón **"Validate"** para comprobar que no haya errores en la configuración del pipeline.

[![p1037.png](https://i.postimg.cc/FRJK9JKs/p1037.png)](https://postimg.cc/ygsKjx9w)
[![p1038.png](https://i.postimg.cc/QCQx7G52/p1038.png)](https://postimg.cc/xNd2rZ65)

#### **Paso 4: Ejecutar el Pipeline**

1. **Ejecutar el Pipeline**: Haz clic en el botón **"Debug"** en la parte superior para ejecutar el pipeline en modo de depuración.

[![p1039.png](https://i.postimg.cc/hPnGBvRc/p1039.png)](https://postimg.cc/z3dNwJfc)

#### **Paso 5: Revisar los Resultados**

1. Verificamos los resultados.

[![p1040.png](https://i.postimg.cc/PrM5rHdZ/p1040.png)](https://postimg.cc/ykkHvwgY)

#### **Paso 6: Verificar la creación de archivos en el ADLS**

1. Nos dirigimos al ADLS y revisamos las rutas **ecdc/hospital_admissions_weekly** y **ecdc/hospital_admissions_daily** para ver si se crearon los archivos `hospital_admissions_weekly.csv` y `hospital_admissions_daily.csv`.

[![p1041.png](https://i.postimg.cc/DfLzHsDx/p1041.png)](https://postimg.cc/DmfFbW7G)
[![p1042.png](https://i.postimg.cc/fRTTVPbP/p1042.png)](https://postimg.cc/LnwM7NLk)
[![p1043.png](https://i.postimg.cc/6pj6nh2p/p1043.png)](https://postimg.cc/hXmqqT1k)
[![p1044.png](https://i.postimg.cc/Xvrj8ydc/p1044.png)](https://postimg.cc/yJzC6xfk)